In [ ]:
%pip install langkit[all]==0.0.24.dev2

Let's say we want the standard LLM metrics for English prompts, but we don't want any metrics on the responses since they're in a different unsupported language. We can use the default configurations for the prompt metrics and set the response metric configurations to `None` to turn them off.


In [2]:
import langkit.llm_metrics as llmm
from langkit import LangKitConfig

no_response_config = LangKitConfig(
    prompt_languages={"en"},
    response_languages={"pt"},
    response_pattern_file_path=None,  # regexes are semi-international, could leave this as default
    sentiment_lexicon=None,  # use multilingual sentiment model for the prompt
    response_sentiment_lexicon=None,
    response_sentiment_model_path=None,
    response_theme_file_path=None,
    response_topic_model_path=None,
    response_toxicity_model_path=None,
    response_transformer_name=None,
)
schema = llmm.init(config=no_response_config)

In [3]:
import whylogs as why
from langkit.whylogs.samples import load_chats, show_first_chat

chats = load_chats()
results = why.log(chats, schema=schema)

for column in results.view().get_columns().keys():
  print(column)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


⚠️ No session found. Call whylogs.init() to initialize a session and authenticate. See https://docs.whylabs.ai/docs/whylabs-whylogs-init for more information.
prompt
response
prompt.has_patterns
prompt.sentiment_multi
prompt.flesch_reading_ease
prompt.automated_readability_index
prompt.aggregate_reading_level
prompt.syllable_count
prompt.lexicon_count
prompt.sentence_count
prompt.character_count
prompt.letter_count
prompt.polysyllable_count
prompt.monosyllable_count
prompt.difficult_words
prompt.jailbreak_similarity
prompt.toxicity


We have a multilingual sentiment model that includes Portuguese, so we can enable it to track response sentiment.

In [4]:
pt_sentiment_config = LangKitConfig(
    prompt_languages={"en"},
    response_languages={"pt"},
    response_pattern_file_path=None,
    sentiment_lexicon=None,  # disable English-only sentiment model
    response_sentiment_lexicon=None,
    #response_sentiment_model_path=None,  # enable (don't disable) multilingual sentiment model
    response_theme_file_path=None,
    response_topic_model_path=None,
    response_toxicity_model_path=None,
    response_transformer_name=None,
)
schema = llmm.init(config=pt_sentiment_config)
results = why.log(chats, schema=schema)

for column in results.view().get_columns().keys():
  print(column)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


prompt
response
prompt.has_patterns
prompt.sentiment_multi
response.sentiment_multi
prompt.flesch_reading_ease
prompt.automated_readability_index
prompt.aggregate_reading_level
prompt.syllable_count
prompt.lexicon_count
prompt.sentence_count
prompt.character_count
prompt.letter_count
prompt.polysyllable_count
prompt.monosyllable_count
prompt.difficult_words
prompt.jailbreak_similarity
prompt.toxicity


The toxicity module supports translation, so we can configure a Portuguese to English translator and track response toxicity with the English-only model.

In [5]:
"""
from transformers import AutoProcessor, SeamlessM4TModel
from langkit.translator import Translator

class Seamless_M4T(Translator):
  def __init__(self, src_lang="eng", tgt_lang="eng"):
    self._processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-medium")
    self._model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-medium")
    self._src_lang = src_lang
    self._tgt_lang = tgt_lang

  def translate(self, text):
    text_inputs = self._processor(text = text, src_lang=self._src_lang, tgt_lang=self._tgt_lang, return_tensors="pt")
    output_tokens = self._model.generate(**text_inputs, tgt_lang=self._tgt_lang, generate_speech=False)
    return self._processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)

print(Seamless_M4T("por", "eng").translate("Olá, o meu cão é lindo."))
"""

from langkit.translator import Translator
class FakeTranslator(Translator):
    def translate(self, text):
        return "Hello, my dog is beautiful"

In [6]:
import langkit.toxicity as tx
tx.RESPONSE_TRANSLATOR = FakeTranslator()  # Seamless_M4T("por", "eng")

translated_config = LangKitConfig(
    prompt_languages={"en"},
    response_languages={"pt"},
    response_pattern_file_path=None,
    sentiment_lexicon=None,
    response_sentiment_lexicon=None,
    response_sentiment_model_path=None,
    response_theme_file_path=None,
    response_topic_model_path=None,
    # response_toxicity_model_path=None,
    response_transformer_name=None,
)
schema = llmm.init(config=translated_config)
results = why.log({"prompt": "Hi, how's your dog?", "response": "Olá, o meu cão é lindo."}, schema=schema)

for column in results.view().get_columns().keys():
  print(column)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


prompt
response
prompt.has_patterns
prompt.sentiment_multi
prompt.flesch_reading_ease
prompt.automated_readability_index
prompt.aggregate_reading_level
prompt.syllable_count
prompt.lexicon_count
prompt.sentence_count
prompt.character_count
prompt.letter_count
prompt.polysyllable_count
prompt.monosyllable_count
prompt.difficult_words
prompt.jailbreak_similarity
prompt.toxicity
response.toxicity
